In [1]:
import pandas as pd
import numpy as np
from scipy import stats
from rl687.policies.Linear_Softmax import LinearSoftmax
import csv
from sklearn.model_selection import train_test_split
import cma
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt

In [2]:
class data_preprocess:
    def __init__(self,data_path,num_batches=5):
        self.path=data_path
        self.num_batches=num_batches
        data=[]
        with open(self.path,newline='',encoding='utf-8') as csv_file:
            file=csv.reader(csv_file,delimiter=',')
            for i, H in enumerate(file):
                data.append(H)
        self.data=np.asarray(data)
        self.data_batches=[]
        self.N=200000
        x=self.N//num_batches
        for i in range(num_batches):
            start=i*x
            end=start+x if start+x<=200000 else 200000
            self.data_batches.append(self.data[start:end])
        self.D_s=[]
        self.D_c=[]
        
    def split_data(self,split):
        for i in range(self.num_batches):
            D_c, D_s = train_test_split(self.data_batches[i],test_size=split)
            self.D_c.append(D_c)
            self.D_s.append(D_s)


        
    

In [3]:
class HCOPE():
    def __init__(self,D_c,theta_b,num_actions,num_features,order,n_Ds,delta,lower_bound):
            self.D_c=D_c
            self.theta_b=theta_b
            self.num_actions=num_actions
            self.num_state_features=num_features
            self.order=order
            self.Ds_size=n_Ds
            self.delta=delta
            self.b=lower_bound
            self.pb=LinearSoftmax(self.num_actions,self.order, self.num_state_features,0.001,self.theta_b,sigma=1)
    
    def PDIS(self,H,pe):
        prev_prod=1
        pdis=0
        for i in range(len(H)//3):
            s=np.float(H[i*3])
            #print(i)
            a=int(H[i*3+1])
            r=np.float(H[i*3+2])
            #print(pe.policy(s,a))
            prod=(float(pe.policy(s,a))/float(self.pb.policy(s,a)))*prev_prod
            pdis+=r*prod
            prev_prod=prod
        return pdis
    
    def thetaTopolicy(self,theta):
        p=LinearSoftmax(self.num_actions,self.order, self.num_state_features,0.001,theta,sigma=1)
        #p.theta=theta
        return p
    
    def estimate_J(self,data,theta_e):
        policy_b=self.thetaTopolicy(self.theta_b)
        policy_e=self.thetaTopolicy(theta_e)
        J=0
        pdis_list=[]
        n=data.shape[0]
        for H in data:
            pdis_H=self.PDIS(H,policy_e)
            J+=pdis_H
            pdis_list.append(pdis_H)
        J/=n
        pdis_list=np.array(pdis_list)
        return J,pdis_list
    
    def ssd(self,J,pdis_list):
        n=len(pdis_list)
        return ((np.sum((pdis_list-J)**2)/(n-1))**(0.5))

    def student_t_test(self,D,theta_e):
        J,PDIS_list=self.estimate_J(D,theta_e)
        std=self.ssd(J,PDIS_list)
        n=len(PDIS_list)
        return (J-((std/n**(0.5))*stats.t.ppf(1-self.delta,n-1)))
    
    def optimise_PDIS(self,theta_e):
        J,PDIS_list=self.estimate_J(self.D_c,theta_e)
        std=self.ssd(J,PDIS_list)
        g=J-2*((std/self.Ds_size**(0.5))*stats.t.ppf(1-self.delta,self.Ds_size-1))
        if g<self.b:
            return 100000
        else: return -J
           

In [4]:
def CMAES_policy(D_c, theta_e,theta_b,num_actions,num_features,order,n_Ds,delta,b):
    policies=[]
    pdis = HCOPE(D_c,theta_b,num_actions,num_features,order,n_Ds,delta,b)
    c=0
    es = cma.CMAEvolutionStrategy(theta_b, 0.7,{'bounds': [-10, 10]})
    #ret = es.optimize(pdis.optimise_PDIS)
    while es.stop() or c<40:
        solutions = es.ask()
        es.tell(solutions, [pdis.optimise_PDIS(x) for x in solutions])
        c+=1
        if es.result[1]<-9:
            policies.append((es.result[0]))
    return policies,es

In [5]:
num_a=2
num_dims=1
order=1
theta_e=np.array([1,1,0.01,-0.01])
theta_b=np.array([0.01,-0.01,1,1])
delta=0.001
b=1

In [6]:
data=data_preprocess('data.csv',4)
data.split_data(0.3)

In [ ]:
policies=[]
best_policies=[]
for i in range(1):
    D_c=data.D_c[i]
    D_s=data.D_s[i]
    n_Ds=len(D_s)
    policy,best_policy=CMAES_policy(D_c, theta_e,theta_b,num_a,num_dims,order,n_Ds,delta,b)
    policies=policies+policy
    best_policies.append(best_policy.result[0])

(4_w,8)-aCMA-ES (mu_w=2.6,w_1=52%) in dimension 4 (seed=308257, Fri Dec 13 18:08:35 2019)


In [ ]:
#policies 
policies2=[[0.6627, 0.9739, -0.9077, -0.9822],
[1,1,-1,-1],
[1.4950253 , 1.38426051,  -2.05568217, -0.69637852],
[10.22528905,8.28499955, -0.93434349, -9.96264375],
[60.97452301,  245.65218064, -216.76817185, -285.62037039 ],[ 71.84634956,   57.20111898,  -35.17542006, -147.88622502],[-1.48810483,-5.8838954,-6.66486254,2.50146831],
[4.04246036,-4.11768224,-1.05767587,4.08253872],
[-3.79293304,-1.92418096,-8.56187103,5.43071975],
[2.14252243,-4.820573,-2.89702609,3.2813018],
[4.12973742,-3.45175617,-0.65938406,4.04142912],
[2.1619701,-2.45386902,-1.03488262,4.34338135],
[2.22497536,-22.64193402,-15.81590115,10.73880119],
[1.13819907,-4.85300822,-2.16992484,1.14013321],
[2.20230796,-4.14083396,-2.24512397,4.02302285],
[0.31500172,8.45417112,-14.55203614,-13.78692092]]       
         
policies=policies+policies2        

In [ ]:
#test on safety dataset
lower_bound=2*1.03
x=[0]*len(policies)
remove_indices=[]
test=HCOPE(data.data,theta_b,num_a,num_dims,order,n_Ds,0.001,b)
for i in range(len(policies)):
    for j in range(1):
        theta_e=np.asarray(policies[i])
        D_s=data.D_s[j]
        score=test.student_t_test(D_s,theta_e)
        if score>lower_bound:
            if score>x[i]:
                x[i]=score
        else: 
            remove_indices.append(i)

for i in remove_indices:
    del policies[i]
    del x[i]

print(sorted(zip(x,policies))
policies=[k for _,k in sorted(zip(x,policies))]


In [ ]:
for i in range(len(policies)):
    with open(str(i+1)+'.csv', "w") as csvfile:
        csvwriter = csv.writer(csvfile,  delimiter=',')
        csvwriter.writerow(list(policies[0]))



In [ ]:
p1=LinearSoftmax(2,1, 1,0.001,theta_b,1)
X=[]
i=0.0001

plt.plot(range(0,1,0.0001),[p1.policy(k,a) for k in X] , 'line type', label='pi_b')
plt.title('title')
plt.ylabel('a=0, probablities')
plt.xlabel('states')
plt.legend()
plt.show()